# Random-forest baseline

In this notebook, we carry out a simple baseline based on RandomForest and kmer profiles. 


## We first import packages

In [ ]:
# generic imports 
import pandas as pd
import os
import numpy as np
import re

# sklearn 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.decomposition import PCA

# seaborn
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# set random seed
np.random.seed(123)

import utils

## 1. Load train dataset

In [ ]:
train_file = "../dataset/datachallenge-traindata.csv"
df_train = pd.read_csv(train_file, sep = ';')
df_train

### Extract sequences

In [ ]:
# extract sequences
seqs = df_train["seq"].values

In [ ]:
# print minimum and maximum sequence length
seq_len = [len(x) for x in seqs]
print("minimum / maximum sequence length = {} / {}".format(np.min(seq_len),np.max(seq_len)))

In [ ]:
# show histogram
sns.histplot(x=seq_len)

## 2. Build kmer profiles

We first build a dictionary associating an index to each kmer. Note that we will only consider kmers made of A, T, G and C's only.

In [ ]:
k = 7
kmer_dic = utils.build_kmer_dic(seqs, k)

We then extract a matrix containing kmer profiles. Each column of the matrix will correspond to a kmer of the dictionary, and will count the number of occurences of this kmer in the sequences.

Note that here we don't need to to padd or truncate sequences.

In [ ]:
X_train = utils.build_kmer_profile_matrix(seqs, k, kmer_dic)
print(X_train.shape)

In [ ]:
print("number of columns of X = {} and number of kmers of the dictionary = {}".format(X_train.shape[1], len(kmer_dic)))
print("min/max value in X = {}/{} ".format(np.min(X_train), np.max(X_train)))

We can plot a PCA to make sure that the matrix is well formed.

In [ ]:
pca = PCA(n_components = 2)
Xpca_train = pca.fit_transform(X_train)
plt.scatter(Xpca_train[:,0], Xpca_train[:,1], alpha = 0.4)

## 3. Fit Random forest model

In [ ]:
# encode train labels 
labEncod = LabelEncoder()
y_train = labEncod.fit_transform(df_train.label)

In [ ]:
# instanciate RF model 
rf = RandomForestClassifier()

# define model parameters 
parameters = {'n_estimators':[100, 200, 300]}

# define grid search paramaters
clf = GridSearchCV(rf, parameters, cv=5, verbose=2)

# fit the model
clf.fit(X_train, y_train)

In [ ]:
clf.cv_results_

In [ ]:
clf.best_params_

## 4. Compute performance on train dataset

In [ ]:
# compute train performance 
pred_train = clf.predict(X_train)
report = classification_report(y_train, pred_train, target_names=labEncod.classes_)
print("\n**** classification report ****")
print(report)

In [ ]:
# compute confusion matrix 
cm = confusion_matrix(y_train, pred_train)
print("\n**** confusion matrix ****")
print(cm)
# compute sensi/speci and macro accuracy 
sensi = cm[0,0]/(cm[0,0]+cm[0,1])
print('Sensitivity: ', sensi )

speci = cm[1,1]/(cm[1,0]+cm[1,1])
print('Specificity: ', speci)

macro_acc = 0.5*(sensi+speci)
print('Macro accuracy: ', macro_acc)